In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('songdata.csv')
df.head(3)


,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...


In [3]:
df.shape

(57650, 4)

In [4]:
df = df.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [5]:
df.shape

(5000, 3)

In [6]:
#cleaing

In [7]:
df['song'][0]

"I Can't Lie"

In [8]:
df['text'][0]

"I must have been a fool  \nTo love you so hard for so long  \nSo much stronger than before  \nBut so much harder to move on  \nAnd now the bitter chill of the winter  \nStill blows through me like a plague  \nOnly to wake up with an empty bed  \nOn a perfect summer day  \n  \nMy world just feels so cold  \nAnd you find yourself walking down  \nThe wrong side of the road  \nOw!  \n  \nI can't lie your on my mind, stuck inside my head  \nI wanna feel your heart beat for me instead, yeah  \nI just die so much inside now that you're not there  \nI wanna feel your heart beat like yesterday  \n  \nI never did my best to express how I really felt  \nAnd now that I know exactly what I want  \nYou found somebody else  \n  \nOoh my world just feels so cold  \nAnd you find yourself walking on  \nThe wrong side of the road  \nOh yeah  \n  \nI can't lie your on my mind stuck inside my head  \nI wanna feel your heart beat for me instead, yeah  \nI just die so much inside now that you're not there  

In [9]:
df['text'] = df['text'].str.lower().replace(r'[^\w\s]','').replace(r'\n',' ', regex=True)

In [10]:
df['text'][0]

"i must have been a fool   to love you so hard for so long   so much stronger than before   but so much harder to move on   and now the bitter chill of the winter   still blows through me like a plague   only to wake up with an empty bed   on a perfect summer day      my world just feels so cold   and you find yourself walking down   the wrong side of the road   ow!      i can't lie your on my mind, stuck inside my head   i wanna feel your heart beat for me instead, yeah   i just die so much inside now that you're not there   i wanna feel your heart beat like yesterday      i never did my best to express how i really felt   and now that i know exactly what i want   you found somebody else      ooh my world just feels so cold   and you find yourself walking on   the wrong side of the road   oh yeah      i can't lie your on my mind stuck inside my head   i wanna feel your heart beat for me instead, yeah   i just die so much inside now that you're not there   i wanna feel your heart beat 

In [11]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [ps.stem(w) for w in tokens]
    return " ".join(stemming)

In [12]:
# tokenization('this is my loving loved lovly ')

In [13]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [14]:
df['text']

0       i must have been a fool to love you so hard fo...
1       all is loneli loneli here for me loneli here f...
2       i can hear her heart beat for a thousand mile ...
3       imagin there 's no countri , it is n't hard to...
4       the night ha a thousand eye and they 're all o...
                              ...                        
4995    nobodi like you everyon left you they 're all ...
4996    [ intro ] la , la , la , la , la , la , la la ...
4997    roll down the gangway so they 'll see that it ...
4998    excus for our money is that religion one more ...
4999    everyday , everyday with you everi littl thing...
Name: text, Length: 5000, dtype: object

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
tfid=TfidfVectorizer(stop_words='english')
matrix=tfid.fit_transform(df['text'])

In [17]:
matrix.shape

(5000, 18046)

In [18]:
similarity=cosine_similarity(matrix)

In [19]:
similarity[0]

array([1.        , 0.01228242, 0.02728304, ..., 0.02007508, 0.00116358,
       0.04550842])

In [20]:
df['song'][0]

"I Can't Lie"

In [21]:
df[df['song']==' ']

,artist,song,text


In [22]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=False,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [25]:
# recommendation('Dance With Me (with Mommy V.)')

In [26]:
import pickle
pickle.dump(df,open('df.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))